<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/nl_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-hjiv38ou
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-hjiv38ou
  Resolved https://github.com/huggingface/transformers to commit b074461ef0f54ce37c5239d30ee960ece28d11ec
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [13]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [14]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [16]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "nl", split="test")
#column_name = "audio, normalized_text"

In [17]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("large")

In [18]:
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:36<00:00, 85.7MiB/s]


In [19]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [20]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="nl")
    return result["text"]

In [21]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [22]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score


In [23]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
count = 0

for sample in voxpopuli_data:
    audio_file_path = sample['audio']['path']
    ground_truth_text = sample['normalized_text']

    # Transcribe the audio
    transcription = transcribe_audio(audio_file_path)

    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    count += 1

    print(f"Processed {count} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")
    # Calculate average metrics
average_wer = total_wer / count
average_precision = total_precision / count
average_recall = total_recall / count
average_f1 = total_f1 / count

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")


Processed 1 files, WER: 0.2, Precision: 0.8333333333333334, Recall: 0.8, F1: 0.816326530612245
Processed 2 files, WER: 0.3076923076923077, Precision: 0.75, Recall: 0.84, F1: 0.7924528301886793
Processed 3 files, WER: 0.13636363636363635, Precision: 0.8529411764705882, Recall: 0.8529411764705882, F1: 0.8529411764705882
Processed 4 files, WER: 0.2, Precision: 0.8, Recall: 0.8, F1: 0.8000000000000002
Processed 5 files, WER: 0.3684210526315789, Precision: 0.625, Recall: 0.5882352941176471, F1: 0.6060606060606061
Processed 6 files, WER: 0.21428571428571427, Precision: 0.8571428571428571, Recall: 0.9230769230769231, F1: 0.888888888888889
Processed 7 files, WER: 0.18421052631578946, Precision: 0.7941176470588235, Recall: 0.7941176470588235, F1: 0.7941176470588235
Processed 8 files, WER: 0.1111111111111111, Precision: 0.9375, Recall: 0.8823529411764706, F1: 0.9090909090909091
Processed 9 files, WER: 0.35, Precision: 0.631578947368421, Recall: 0.6666666666666666, F1: 0.6486486486486486
Processe